In [1]:
import pandas as pd
from generator_mamdani import MamdaniRuleBaseGenerator
from rete_network import ReteNetwork
from anfis import ANFIS
import random

In [14]:
weight = {
    "x1": 0.154,
    "x2": 0.225,
    "x3": 0.354,
    "x4": 0.043,
    "x5": 0.124,
    "x6": 0.1,
}

risk_reducing = {"x4", "x5", "x6"}

gen = MamdaniRuleBaseGenerator() 
all_rules = gen.generate_rules()
random.shuffle(all_rules)
#all_rules = all_rules[:1000]

split_idx = int(0.8 * len(all_rules))
train_rules = all_rules[:split_idx]
test_rules = all_rules[split_idx:]

rete_train = ReteNetwork(train_rules)
anfis = ANFIS(rete_train, poly_degree=2)
anfis.initialize_coeffs(weight, risk_reducing)

In [3]:
anfis.train_model(lr=0.01, epochs=50, batch_size=128)

Epoch 1/50, MSE=0.004280
Epoch 2/50, MSE=0.000852
Epoch 3/50, MSE=0.000371
Epoch 4/50, MSE=0.000208
Epoch 5/50, MSE=0.000135
Epoch 6/50, MSE=0.000101
Epoch 7/50, MSE=0.000081
Epoch 8/50, MSE=0.000069
Epoch 9/50, MSE=0.000059
Epoch 10/50, MSE=0.000049
Epoch 11/50, MSE=0.000042
Epoch 12/50, MSE=0.000036
Epoch 13/50, MSE=0.000030
Epoch 14/50, MSE=0.000026
Epoch 15/50, MSE=0.000022
Epoch 16/50, MSE=0.000019
Epoch 17/50, MSE=0.000017
Epoch 18/50, MSE=0.000015
Epoch 19/50, MSE=0.000013
Epoch 20/50, MSE=0.000012
Epoch 21/50, MSE=0.000010
Epoch 22/50, MSE=0.000009
Epoch 23/50, MSE=0.000008
Epoch 24/50, MSE=0.000008
Epoch 25/50, MSE=0.000007
Epoch 26/50, MSE=0.000007
Epoch 27/50, MSE=0.000006
Epoch 28/50, MSE=0.000006
Epoch 29/50, MSE=0.000005
Epoch 30/50, MSE=0.000005
Epoch 31/50, MSE=0.000005
Epoch 32/50, MSE=0.000005
Epoch 33/50, MSE=0.000005
Epoch 34/50, MSE=0.000005
Epoch 35/50, MSE=0.000004
Epoch 36/50, MSE=0.000004
Epoch 37/50, MSE=0.000004
Epoch 38/50, MSE=0.000004
Epoch 39/50, MSE=0.00

In [4]:
anfis.save_model("anfis_b_p2.pkl")

Model saved in anfis_b_p2.pkl


In [5]:
rete_test = ReteNetwork(test_rules)
#anfis.load_model("anfis_base.pkl")
results = anfis.check_model_adequacy(rete_test)
print(results)

{'MAE': 0.0009193257428705692, 'RMSE': 0.0014879542868584394, 'Correlation': 0.9999409914016724}


In [6]:
anfis.get_global_polynomial_equation()

'R = sigmoid(-0.2237 + 0.1618*x1 + 0.2626*x2 + 0.4912*x3 + -0.1837*x4 + -0.3031*x5 + -0.2668*x6 + 0.0666*x1^2 + 0.1360*x2^2 + 0.1904*x3^2 + -0.0846*x4^2 + -0.1409*x5^2 + -0.1233*x6^2 + 0.0854*x1*x2 + 0.1173*x1*x3 + 0.0115*x1*x4 + -0.0132*x1*x5 + -0.0055*x1*x6 + 0.1185*x2*x3 + 0.0344*x2*x4 + 0.0069*x2*x5 + 0.0173*x2*x6 + 0.1044*x3*x4 + 0.0867*x3*x5 + 0.0907*x3*x6 + -0.1165*x4*x5 + -0.1098*x4*x6 + -0.1314*x5*x6)'

In [15]:
anfis.train_and_reduce(
    initial_epochs=30, 
    batch_size=128, 
    reduction_epochs=30, 
    significance_threshold=0.05,
    load_filepath="anfis_b_p2.pkl"
)

Model loaded from anfis_b_p2.pkl

Model reduction, round 1
Global equation:
R = sigmoid(-0.2237 + 0.1618*x1 + 0.2626*x2 + 0.4912*x3 + -0.1837*x4 + -0.3031*x5 + -0.2668*x6 + 0.0666*x1^2 + 0.1360*x2^2 + 0.1904*x3^2 + -0.0846*x4^2 + -0.1409*x5^2 + -0.1233*x6^2 + 0.0854*x1*x2 + 0.1173*x1*x3 + 0.0115*x1*x4 + -0.0132*x1*x5 + -0.0055*x1*x6 + 0.1185*x2*x3 + 0.0344*x2*x4 + 0.0069*x2*x5 + 0.0173*x2*x6 + 0.1044*x3*x4 + 0.0867*x3*x5 + 0.0907*x3*x6 + -0.1165*x4*x5 + -0.1098*x4*x6 + -0.1314*x5*x6)
Removing coefficients with indices: [15, 16, 17, 19, 20, 21] (|value| < 0.05)
Epoch 1/30, MSE=0.000033
Epoch 2/30, MSE=0.000004
Epoch 3/30, MSE=0.000002
Epoch 4/30, MSE=0.000002
Epoch 5/30, MSE=0.000002
Epoch 6/30, MSE=0.000001
Epoch 7/30, MSE=0.000001
Epoch 8/30, MSE=0.000001
Epoch 9/30, MSE=0.000001
Epoch 10/30, MSE=0.000001
Epoch 11/30, MSE=0.000001
Epoch 12/30, MSE=0.000001
Epoch 13/30, MSE=0.000001
Epoch 14/30, MSE=0.000001
Epoch 15/30, MSE=0.000001
Epoch 16/30, MSE=0.000001
Epoch 17/30, MSE=0.000001


In [16]:
anfis.save_model("anfis_f_p2.pkl")

Model saved in anfis_f_p2.pkl


In [17]:
results = anfis.check_model_adequacy(rete_test)
print(results)
anfis.get_global_polynomial_equation()

{'MAE': 0.0004022665962111205, 'RMSE': 0.000581269443500787, 'Correlation': 0.9999910593032837}


'R = sigmoid(-0.2328 + 0.1708*x1 + 0.2794*x2 + 0.5045*x3 + -0.1809*x4 + -0.3053*x5 + -0.2673*x6 + 0.0713*x1^2 + 0.1506*x2^2 + 0.1928*x3^2 + -0.0777*x4^2 + -0.1370*x5^2 + -0.1188*x6^2 + 0.0888*x1*x2 + 0.1133*x1*x3 + 0.1173*x2*x3 + 0.1095*x3*x4 + 0.0901*x3*x5 + 0.0947*x3*x6 + -0.1154*x4*x5 + -0.1074*x4*x6 + -0.1303*x5*x6)'

In [19]:
anfis = ANFIS(None,poly_degree=2)
anfis.load_model("anfis_f_p2.pkl")
test_samples = [
    {"x1": 0.1, "x2": 0.15, "x3": 0.1, "x4": 0.97, "x5": 0.15, "x6": 0.15},
    {"x1": 0.1, "x2": 0.15, "x3": 0.1, "x4": 0.97, "x5": 0.15, "x6": 0.42},
    {"x1": 0.1, "x2": 0.15, "x3": 0.1, "x4": 0.97, "x5": 0.15, "x6": 0.55},
    {"x1": 0.1, "x2": 0.15, "x3": 0.1, "x4": 0.97, "x5": 0.15, "x6": 0.87},
]

for i, sample in enumerate(test_samples, start=1):
    y_pred = anfis.predict(sample)
    print(f"Sample {i}: input={sample}, predict={y_pred:.4f}")

Model loaded from anfis_f_p2.pkl
Sample 1: input={'x1': 0.1, 'x2': 0.15, 'x3': 0.1, 'x4': 0.97, 'x5': 0.15, 'x6': 0.15}, predict=0.2762
Sample 2: input={'x1': 0.1, 'x2': 0.15, 'x3': 0.1, 'x4': 0.97, 'x5': 0.15, 'x6': 0.42}, predict=0.2486
Sample 3: input={'x1': 0.1, 'x2': 0.15, 'x3': 0.1, 'x4': 0.97, 'x5': 0.15, 'x6': 0.55}, predict=0.2353
Sample 4: input={'x1': 0.1, 'x2': 0.15, 'x3': 0.1, 'x4': 0.97, 'x5': 0.15, 'x6': 0.87}, predict=0.2034
